<a href="https://colab.research.google.com/github/manualrg/dslab-nlp-dl/blob/master/04_intronlp_spacy_ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SpaCy. uso general y extracción de entidades

`SpaCy` es una biblioteca de código abierto para el procesamiento avanzado del lenguaje natural (PLN) en Python. El concepto fundamental es la `pipeline`:

[Language Processing Pipelines](https://spacy.io/usage/processing-pipelines)

La misma encadena las accciones que se aplican al texto de manera automática  por el modelo elegido (components). Para trabajar con SpaCy, es necesario descargar un modelo pre-entrenado (generalmente, tienen 3 tamaños):

[SpaCy Models. Spanish](https://spacy.io/models/es)

In [1]:
!python -m spacy download es_core_news_md
!python -m spacy download es_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 16.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 3.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import pandas as pd
import spacy
from spacy import displacy

# Introducción

In [3]:
nlp = spacy.load("es_core_news_md")  # cargar el modelo de spacy, generalmente, en la variable `nlp`
nlp.pipe_names  # Built-in pipeline components

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

Se pueden eliminar algunos pasos del componente para hacer el procesamiento más rapido

In [4]:
nlp_fast = spacy.load("es_core_news_md", exclude=["tok2vec", "lemmatizer"])
nlp_fast.pipe_names

['morphologizer', 'parser', 'attribute_ruler', 'ner']

# Procesamiento

El elemento de entrada es siempre un `texto` y el de salida un `SpaCy Doc`. El mismo, tiene una representación como `str` pero por debajo, ha sido transformado

Cada palabra ha sido `tokenizada` y de la misma se puede extraer información lingüistica [Linguistic Features](https://spacy.io/usage/linguistic-features)

In [5]:
text = text= "La chaqueta impermeable Columbia Watertight II para hombre\
 está fabricada con tejido 100% poliéster y cuenta con tecnología Omni-Tech™\
 , que ofrece protección contra la lluvia y transpirabilidad.\
 Disponible en color negro y en tallas S, M, L y XL. \
 Fantástica para actividades al aire libre como senderismo y camping. \
 Precio: $89.99."

doc = nlp(text)
type(doc)

spacy.tokens.doc.Doc

In [6]:
# La lemmatizacion en spaCy, por defecto incluye el analisis de PoS de cada token
# haciendo que sea mas lenta, pero obteniedo tokens mas reconocibles
for token in doc[:5]:
  print(token.text, token.pos_, token.is_stop, token.lemma_)

La DET True el
chaqueta NOUN False chaqueta
impermeable ADJ False impermeable
Columbia PROPN False Columbia
Watertight PROPN False Watertight


In [7]:
for token in nlp("Las chaquetas son impermeables y unisex"):
  print(token.text, token.pos_, token.is_stop, token.lemma_)

Las DET True el
chaquetas NOUN False chaqueta
son AUX True ser
impermeables ADJ False impermeable
y CCONJ True y
unisex NOUN False unisex


# Herramientas de visualización

SpaCy constituye todo un ecosistema, un ejemplo son las herramientas de visualización [Visualizers ](https://spacy.io/usage/visualizers)


In [8]:
displacy.render(
    nlp("Las chaquetas son impermeables y están de oferta"),
    style="dep", jupyter=True
    )

# Extraccion de Entidades (ER, NER)

Es uno de los componentes que hace especial a spaCy. Consta de un modelo pre-entrenado que analiza por tokens y `spans` para etiquetar las ER.

[EntityRecognizer](https://spacy.io/api/entityrecognizer)

Sobre todo en idiomas fuera del inglés, el tamaño del modelo influye mucho en la calidad de los resultados

In [9]:
displacy.render(
    doc,
    style="ent", jupyter=True
    )

In [10]:
text1 = "From: manuelrg@gmail.com \
Yo, Manuel Romero, con DNI 11123334A deseo darme de baja de la póliza H123456 \
relativa al vehiculo Toyota Auris de matricula FFF-123456 antes del 2025-05-30, \
dado que la cuota de 325 € me parece muy cara. \
Muchas gracias"


In [11]:
doc1_md = nlp(text1)
displacy.render(
    doc1_md,
    style="ent", jupyter=True
    )

In [12]:
nlp_lg = spacy.load("es_core_news_lg")
doc1_lg = nlp_lg(text1)
displacy.render(
    doc1_lg,
    style="ent", jupyter=True
    )

In [13]:
for token in doc[:5]:
  print(token.text, token.ent_iob_, token.ent_type_, token.lemma_, token.like_email, token.like_url)

La O  el False False
chaqueta O  chaqueta False False
impermeable O  impermeable False False
Columbia B MISC Columbia False False
Watertight I MISC Watertight False False


Etiquetado de entidades:
Téngase en cuenta, que una entidad puede extenderse a través de varios `Tokens` (en `spaCy` se denominan  `span`) por lo que la nomenclatura que se sigue es `IOB`:
* I: Inside
* O: Outside (no está en la ninguna entidad)
* B: Begin

PAra mejorar los resultados, se pueden incluir entidades definidas mediante patrón con el componente [SourceEntityRuler ](https://spacy.io/api/entityruler)

In [14]:
ruler = nlp_lg.add_pipe("entity_ruler", before="ner")

patterns = [
                {"label": "ID",
                 "pattern": [
                     {"TEXT": {"REGEX": r"^\d{8}[-]?[A-Z]$"}}
                     ]}
            ]
ruler.add_patterns(patterns)

doc1_lg_rul = nlp_lg(text1)
for ent in doc1_lg_rul.ents:  # acceso directo a las entidades
    print(ent.text, ent.label_)

From: manuelrg@gmail.com Yo MISC
Manuel Romero PER
11123334A ID
H123456 MISC
Toyota Auris MISC
FFF-123456 PER


Esta característica es muy importante, puesto que permite combinar modelos con reglas para darle una cobertura total al proyecto

# Vector Embeddings

Como característica adicional a destacar, spaCy también es capaz de aplicar W2V a las tokens, spans y docs y calcular cómo de similares son

Es importante notar, que  para spans y docs, se calcula el w2v de cada término y luego se promedia

In [15]:
for token in doc1_md[:5]:
  print(token.text, token.has_vector, token.vector_norm, token.vector.shape[0])

From True 29.942974 300
: True 59.92712 300
manuelrg@gmail.com False 0.0 300
Yo True 89.935455 300
, True 33.25681 300


In [16]:
w2v_doc1  = [(token.text, token.has_vector, token.vector_norm, token.vector.shape[0]) for token in doc1_md]

_cols = ["token", "has_vector", "vector_norm", "vector_dim"]
df_w2v_doc1 = pd.DataFrame(data=w2v_doc1, columns=_cols)
df_w2v_doc1.head(10)

,token,has_vector,vector_norm,vector_dim
0,From,True,29.942974,300
1,:,True,59.927120,300
2,manuelrg@gmail.com,False,0.000000,300
3,Yo,True,89.935455,300
4,",",True,33.256809,300
5,Manuel,True,31.905590,300
6,Romero,True,23.671192,300
7,",",True,33.256809,300
8,con,True,42.938641,300
9,DNI,True,57.838348,300


In [17]:
doc_q = nlp("El hidrogeno es un gas noble, \cuyo átomo al menos se compone de \
un protón y un electrón")
doc_k = nlp("El gas natural es el combustible usado en centrales de ciclo \
combinado, siendo una molécula compuesta por un átomo de carbono y 4 de \
hidrógeno")

# Similarity of two documents
print(f"doc_q vs doc_k: {doc_q.similarity(doc_k)}")

doc_q vs doc_k: 0.7614282965660095


In [18]:
doc_k = nlp("España ha ganado la prueba de 4x400 femenino en el mundial de \
relevos, sorprendiendo a la aficion")

# Similarity of two documents
print(f"doc_q vs doc_k: {doc_q.similarity(doc_k)}")

doc_q vs doc_k: 0.4006810784339905


# Ejercicio

1. Crear una funcion que sustituya o elimine las entidades extraidas del texto mediante spaCy

```<MISC>, <PER>, con DNI <ID> deseo darme de baja de la póliza <MISC> relativa al vehiculo <MISC> de matricula <PER> antes del 2025-05-30, dado que la cuota de 325 € me parece muy cara. Muchas gracias```

2. Realizar el ejercicio de extracción y en enmascarado mediante Prompt Engineering mediante IA Generativa